# Monocular Depth Estimation

Based on https://github.com/compphoto/BoostingMonocularDepth

Created by [Artem Konevskikh](https://github.com/artem-konevskikh)

In [ ]:
#@title Install
#@markdown This cell will download all the needed files
# Clone git repo
!git clone https://github.com/compphoto/BoostingMonocularDepth.git

# !wget https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth
!gdown https://drive.google.com/u/0/uc?id=1cU2y-kMbt0Sf00Ns4CN2oO9qPJ8BensP&export=download

# Downloading merge model weights
!mkdir -p /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/
!mv latest_net_G.pth /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/

# Downloading Midas weights
!gdown https://drive.google.com/uc?id=1nqW_Hwj86kslfsXR7EnXpEWdO2csz1cC
!mv model.pt /content/BoostingMonocularDepth/midas/

# # Downloading LeRes weights
!wget https://cloudstor.aarnet.edu.au/plus/s/lTIJF4vrvHCAI31/download
!mv download /content/BoostingMonocularDepth/res101.pth

!git clone https://github.com/CapObvios/Depth-Map-Visualizer.git

!mkdir -p /content/inputs
!mkdir -p /content/outputs
!mkdir -p /content/3d-models

In [ ]:
%cd BoostingMonocularDepth
import glob
#@title Run
#@markdown 1. Put your images to `inputs` folder
#@markdown 2. Select the network model to reconstruct depth map. `LeRes` gives beter resolution, while `MiDas` works faster
depth_net = "LeRes" #@param ["LeRes","MiDas"]
#@markdown 3. Check this if you want to generate 3d model from the obtained depth map. Note that you can achive much better results if you do it manually in 3d editing software
make3d = True #@param {type:"boolean"}

model = 2 if depth_net == "LeRes" else 0

!python run.py --Final --data_dir /content/inputs --output_dir  /content/outputs/ --depthNet $model

if make3d:
  for f in glob.glob('/content/inputs/*.*'):
    !echo $f
    model_name = f.split('/')[-1].split('.')[0]
    texture_name = f.split('/')[-1]
    model_path = '/content/3d-models/'+ model_name + '/'
    depth_map = '/content/outputs/'+model_name+'.png'
    # mtl_path = model_path + 'model.mtl'
    # obj_path = model_path + 'model.obj'
    !mkdir -p $model_path
    !cp $f $model_path
    !cp $depth_map $model_path
    %cd $model_path
    !python /content/Depth-Map-Visualizer/DepthToObj.py --depthPath $depth_map --texturePath $texture_name 
    # --mtlPath $mtl_path --objPath $obj_path



In [ ]:
#@title Export models
#@markdown This will zip 3d models folder, so it will be easier to download the results
%cd /content/
!zip -r 3d-models.zip 3d-models/